In [7]:
using DynamicPolynomials

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=0#ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=0


In [8]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [9]:

k=2

println("Relaxation order: k=",k)

Relaxation order: k=2


In [10]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Condition number: c = 28.583211855912904
  Number of blocks: omega=2
  Size of the largest block: s^max=21
  Number of equality trace constraints: zeta=127
Modeling time:
  0.102066 seconds (95.60 k allocations: 5.447 MiB)
iter=1   val=-3.622241093070757   gap=0.1428438202195215   feas=1.6996335047742301
iter=2   val=-2.1279744308672206   gap=0.07103012219406095   feas=1.0401230850171874
iter=4   val=-2.5706000161272327   gap=0.05585928192288351   feas=0.48698114188346214
iter=8   val=-2.508123145368387   gap=0.03573124308156814   feas=0.15027035075770678
iter=16   val=-2.4582074199697757   gap=0.0003274760332287779   feas=0.02391113337133294
iter=32   val=-2.46401710762825   gap=0.0019363348592739654   feas=0.005471787841456375
iter=64   val=-2.4684717518196484   gap=0.0004628991771342869   feas=0.001386528569020017
iter=76   val=-2.4688937630265513   gap=0.00033443261026976323   feas=0.0009856303280674757
tol sat

-2.4688937630265513

In [11]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

**SumOfSquares+Mosek:
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 126             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : 

-2.4699397871534594

In [12]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

-2.469939806631432